In [18]:
import nltk

In [19]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
from nltk.tokenize import word_tokenize

In [21]:
words = word_tokenize("exists x forall y forall z ((P(y) -> Q(z)) -> (P(x) -> Q(x)))")
print(words)

['exists', 'x', 'forall', 'y', 'forall', 'z', '(', '(', 'P', '(', 'y', ')', '-', '>', 'Q', '(', 'z', ')', ')', '-', '>', '(', 'P', '(', 'x', ')', '-', '>', 'Q', '(', 'x', ')', ')', ')']


In [22]:
# elimenate implication
i = 0
while i < len(words):
    if words[i] == '-':
        words[i] = "or"
        del words[i+1]
        a = i-1
        flag = False
        close = -1
        while a >= 0:
            if words[a] == ')':
                if flag == False:
                  close += 2
                  flag = True
                else:
                  close+=1

            elif words[a] == '(':
                if close == 0:
                   words.insert(a+1,"not")
                   break
                close -= 1

            elif close==0:
                    words.insert(a, "not")
                    break
            a -= 1
    i += 1
ans = ""
i = 0
while i < len(words):
  if words[i]=="P" or words[i]=="Q":
    ans+= words[i]+words[i+1]+words[i+2]+words[i+3]+" "
    i+=4
    continue
  ans+=words[i] + " "
  i+=1

print(ans)

exists x forall y forall z ( not ( not P(y) or Q(z) ) or ( not P(x) or Q(x) ) ) 


In [23]:
import re
def move_quantifiers_left(formula):
    quantifiers = re.findall(r'(forall|exists)\s+(\w+)\s*', formula)
    quantifiers = sorted(quantifiers, key=lambda x: x[1])
    prenex = ''.join([f'{q[0]} {q[1]} ' for q in quantifiers])
    formula = re.sub(r'(forall|exists)\s+(\w+)\s*', '', formula)
    return f'{prenex}{formula}'

2. Move negation inward (Demorgan Law)
3. Remove double-no

In [24]:
words = word_tokenize(ans)
def move_negation_inward(words):
    i = 0

    while i < len(words):
        notMult = False
        if words[i] == "not":
            j = i + 1
            flag = False
            open_parentheses = 0
            while j < len(words):
                if (flag == True) and (open_parentheses == 0):
                  j = i
                  break
                if words[j] == "(":
                    flag = True
                    open_parentheses += 1
                elif words[j] == ")":
                    open_parentheses -= 1
                if words[j] == "not" and open_parentheses == 1:
                   notMult = True
                   words[j] = "not not"
                elif (words[j] == "and" ) and open_parentheses == 1:
                    notMult = True
                    words[j]=="or"
                elif words[j] == "or" and open_parentheses == 1:
                    notMult = True
                    words[j] = "and"
                elif (words[j] == "Q" or words[j] == "P") and open_parentheses == 1:
                    if words[j-1]!= "not not":
                         notMult = True
                         words[j] = " not " + words[j]
                j += 1
        if notMult:
            del words[i]
            i = j+1
        else:
            i += 1
    return words

# Test case
words = ['exists', 'x', 'forall', 'y', 'forall', 'z', '(', 'not', '(', 'not', 'P', '(', 'y', ')', 'or', 'Q', '(', 'z', ')', ')', 'or', '(', 'not', 'P', '(', 'x', ')', 'or', 'Q', '(', 'x', ')', ')', ')']
move_negation_inward(words)
ans = ""
i = 0
while i < len(words):
  if words[i]=="P" or words[i]=="Q":
    ans+= words[i]+words[i+1]+words[i+2]+words[i+3]+" "
    i+=4
    continue
  ans+=words[i] + " "
  i+=1
# p incorrect
print(ans)


exists x forall y forall z ( ( not not P(y) and  not Q ( z ) ) or ( not P(x) or Q(x) ) ) 


 Remove double-not.

In [25]:
def remove_double_not(words):
  words.remove("not not")

remove_double_not(words)
ans = ""
i = 0
# edit the text (text processing)
while i < len(words):
  if words[i]=="P" or words[i]=="Q":
    ans+= words[i]+words[i+1]+words[i+2]+words[i+3]+" "
    i+=4
    continue
  ans+=words[i] + " "
  i+=1
# p incorrect
print(ans)

exists x forall y forall z ( ( P(y) and  not Q ( z ) ) or ( not P(x) or Q(x) ) ) 


In [26]:
from ctypes import string_at
# split quantifiers from the formula
def move_quantifiers_left(string):
    quantifiers = re.findall(r'(forall|exists)\s+(\w+)\s*', string)
    print(quantifiers)
    quantifiers = sorted(quantifiers, key=lambda x: x[1])
    print(quantifiers)
    prenex = ''.join([f'{q[0]} {q[1]} ' for q in quantifiers])
    print(prenex)
    string= re.sub(r'(forall|exists)\s+(\w+)\s*', '', string)
    print(string)
    return f'{prenex}{string}'
move_quantifiers_left(ans)

[('exists', 'x'), ('forall', 'y'), ('forall', 'z')]
[('exists', 'x'), ('forall', 'y'), ('forall', 'z')]
exists x forall y forall z 
( ( P(y) and  not Q ( z ) ) or ( not P(x) or Q(x) ) ) 


'exists x forall y forall z ( ( P(y) and  not Q ( z ) ) or ( not P(x) or Q(x) ) ) '

In [27]:
from collections import defaultdict

# helper function to check if the word is variable
def IsVar(word):
    pattern = r'^[a-zA-Z]+$'
    return  re.match(pattern, word)

# helper function to get the variable name inside the ()
def findvar(word):
    pattern = r'\b([a-zA-Z]+)\(([a-zA-Z]+)\)'
    # Find matches of the pattern in the word
    matches = re.findall(pattern, word)
    variables = []
    for match in matches:
        variables.append(match[1])
    if not variables:
        mat = None
    else:
        mat = variables[0]
    return mat


# looping for the formula
def stand(formula):
    standrizedFormula = ''
    words = formula.split()
    variable_map = defaultdict(list)
    index = 0
    for i, word in enumerate(words):
        if word == "forall" or word =="exists":
            # if the map is not empty
            if variable_map:
                key_to_check = words[i+1]
                # Check if the variable exists in the map
                if key_to_check in variable_map:
                    index+=1
            standrizedFormula+= (word + ' ')
        elif (IsVar(word)) and (word != "and" and word != "or" and "not") :
            # print(word)
            variable_map[f'{word}'].append(index)
            if index > 0 :
                newVar = word + (f'{index}')
            else:
                newVar = word
            standrizedFormula+= (newVar + ' ')

        elif (findvar(word)) and (word != "and" and word != "or"):
            # print(findvar(word))
            vaR = findvar(word)
            variable_map[f'{vaR}'].append(index)
            if index > 0 :
                # vaR[-1] because if the variable was more than 1 letter
                # i made sure that i will always append in the last letter
                letterindex = word.index(vaR[-1]) + 1
                newVar = word[:letterindex] + (f'{index}') + word[letterindex:]

            else:
                newVar = word
            standrizedFormula+= (newVar + ' ')
        else:
            standrizedFormula+= word + ' '

    # print(standrizedFormula)
    return standrizedFormula

formula = "forall x exists y (P(x) and Q(y)) and exists y R(y)"
sentence = "This is a sample sentence."
standformula = stand(formula)
print(standformula)

forall x exists y (P(x) and Q(y)) and exists y1 R(y1) 


In [28]:
def eliminate_universal_quantifiers(formula):
    # Find all universal quantifiers and their variables
    universal_quantifiers = re.findall(r'forall\s+(\w+)\s*', formula)

    # Replace each universally quantified variable with a Skolem constant
    skolem_constants = ['a', 'b', 'c', 'd', 'e']  # Use a list of Skolem constants for substitution
    skolem_mapping = dict(zip(universal_quantifiers, skolem_constants))
    for variable, constant in skolem_mapping.items():
        formula = re.sub(r'\b' + re.escape(variable) + r'\b', constant, formula)

    # Remove universal quantifiers from the formula
    formula = re.sub(r'forall\s+(\w+)\s*', '', formula)

    return formula

formula = "forall x (P(x) ∧ Q(y))"
result = eliminate_universal_quantifiers(formula)
print("Formula after eliminating universal quantifiers:", result)

formula = "exists x forall y forall z ( ( P(y) and  not Q ( z ) ) or ( not P(x) or Q(x) ) ) "
result = eliminate_universal_quantifiers(formula)
print("Formula after eliminating universal quantifiers:", result)

Formula after eliminating universal quantifiers: (P(a) ∧ Q(y))
Formula after eliminating universal quantifiers: exists x ( ( P(a) and  not Q ( b ) ) or ( not P(x) or Q(x) ) ) 


skolization

In [29]:
import random
import string
tokens = ['exists', 'x', 'forall', 'y', 'forall', 'z', '(', 'not', '(', 'not', 'P', '(', 'y', ')', 'or', 'Q', '(', 'z', ')', ')', 'or', '(', 'not', 'P', '(', 'x', ')', 'or', 'Q', '(', 'x', ')', ')', ')']


def skolemizationExists(tk):
    def generateRandomLetters(varia,listoffun):
        letter = random.choice(string.ascii_lowercase)
        while letter in varia and letter in listoffun:
            if len(varia) != 26 and len(listoffun) !=26 :
                letter = random.choice(string.ascii_lowercase)
            else:
                for i in range(5):
                    letter += random.choice(string.ascii_lowercase)
        return letter

    def detectVariables(rule):
        i = 0
        listofvariables = []
        while i < len(rule):
            if rule[i] == "exists" or rule[i] == "forall":
                listofvariables.append(rule[i+1])
            i+=1
        return listofvariables

    def detectFunctions(rule):
        i = 0
        functionList = []
        listofvar = detectVariables(rule)
        while i < len(rule):
            if rule[i] == '(' and ( rule[i-1] != 'not' and rule[i-1] != 'and' and rule[i-1] != 'or' ) and (rule[i-1] not in listofvar):
                functionList.append(rule[i-1])
            i+=1
        print(functionList)
        return functionList

    varlist = []
    def existvariable():
        i = 0
        while i < len(tk):
            if tk[i] == "exists":
                varlist.append(tk[i+1])
            i+=1

    existvariable()
    listOfFunctions = detectFunctions(tk)
    i = 0
    newL = generateRandomLetters(varlist,listOfFunctions)
    newSentence = ''
    while i < len(tk):
        if tk[i] == "exists":
            i+=1
        elif tk[i] in varlist:
            newSentence+= newL + '(' + f'{tk[i]}' + ')'
        else:
            newSentence+=tk[i]+' '
        i+=1

    return newSentence
words = word_tokenize(result)
answer = skolemizationExists(words)
print(answer)

['(', 'P', 'Q', 'P', 'Q']
( ( P ( a ) and not Q ( b ) ) or ( not P ( o(x)) or Q ( o(x)) ) ) 


In [30]:
def get_clauses(expression):
    # Remove unnecessary spaces
    expression = expression.replace("(", "").replace(")", "").strip()

    # Split the expression into its clauses based on the 'or' operator
    clauses = expression.split(' or ')

    # Remove any leading or trailing spaces from each clause
    clauses = [clause.strip() for clause in clauses]

    # Split each clause based on the 'and' operator
    split_clauses = []
    for clause in clauses:
        split_clause = clause.split(' and ')
        split_clauses.extend(split_clause)

    # Remove any leading or trailing spaces from each split clause
    split_clauses = [clause.strip() for clause in split_clauses]

    return split_clauses

# Given expression
expression = "( ( P ( a ) and not Q ( b ) ) or ( not P ( f(x)) or Q ( f(x)) ) )"

# Extract clauses from the expression
clauses = get_clauses(expression)

# Print the list of clauses
print("Clauses:", clauses)

Clauses: ['P  a', 'not Q  b', 'not P  fx', 'Q  fx']


In [31]:
from nltk.tokenize import word_tokenize

def getfunction(clauses):
    list_of_functions = {'P': 0, 'Q': 0}

    # Loop through the clauses and count occurrences of 'P' and 'Q'
    for clause in clauses:
        tokens = word_tokenize(clause)
        for i in range(len(tokens)):
            if tokens[i] == 'not':
                if tokens[i + 1] in list_of_functions:
                    list_of_functions[tokens[i + 1]] -= 1
                break
            elif tokens[i] in list_of_functions:
                list_of_functions[tokens[i]] += 1
                break

    return list_of_functions

ans = getfunction(clauses)
flag = True

# Iterate over the items of the dictionary
for item, val in ans.items():
    if val != 0:
        flag = False

if flag:
    print("Null")
else:
    print("No Solution")

Null


In [32]:
# not important for now
result_string = ', '.join(clauses)
words = word_tokenize(result_string)
# print(words)
def generateRandomLetters(listoffun):
    letter = random.choice(string.ascii_lowercase)
    while letter in listoffun:
        if len(listoffun) !=26 :
            letter = random.choice(string.ascii_lowercase)
        else:
            for i in range(5):
                letter += random.choice(string.ascii_lowercase)
    return letter

def changeNames(sent):
  newsent = ''
  varlist = []
  for i in sent:
    if i in varlist:
      newL = generateRandomLetters(varlist)
      newsent+= newL+' '
      varlist.append(newL)
    elif i == 'not'or i == ',':
      newsent+= i + ' '
    else :
      varlist.append(i)
      newsent+= i + ' '
  print(newsent)

changeNames(words)

P a , not Q b , not u fx , o c 
